In [1]:
import random
import sys
import inspect
from os.path import dirname
from pprint import pprint
from CybORG import CybORG
from CybORG.Simulator.Scenarios.CFileReaderScenarioGenerator import FileReaderScenarioGenerator
from CybORG.reproduction_env import CybORGExtension

In [2]:
dic = inspect.getfile(CybORG)
scenario_path = dirname(dic) + f'/Simulator/Scenarios/scenario_files/ThreeWayScenario.yaml'
vulhost_path = dirname(dic) + f'/Simulator/Scenarios/scenario_files/abstractvulnerability/test.yaml'
sg = FileReaderScenarioGenerator(scenario_path)
env = CybORG(sg)

e:\Anaconda3\envs\cyborg\lib\site-packages\gym\utils\seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [35]:
env.environment_controller.state.np_random.random()

0.95885872923095

In [3]:
extension = CybORGExtension(env)
extension.load_host_absvul_map_from_yaml(vulhost_path)

In [4]:
env.environment_controller.state.hosts

{'Defender': <CybORG.Simulator.Host.Host at 0x285b715dca0>,
 'API_Gateway': <CybORG.Simulator.Host.Host at 0x285b715dd60>,
 'ServiceA1': <CybORG.Simulator.Host.Host at 0x285b71b5340>,
 'ServiceA2': <CybORG.Simulator.Host.Host at 0x285b71b5130>,
 'ServiceB1': <CybORG.Simulator.Host.Host at 0x285b71b5ac0>,
 'ServiceB2': <CybORG.Simulator.Host.Host at 0x285b71b55e0>,
 'ServiceC1': <CybORG.Simulator.Host.Host at 0x285b71c1a30>,
 'ServiceC2': <CybORG.Simulator.Host.Host at 0x285b71c14c0>,
 'DatabaseA': <CybORG.Simulator.Host.Host at 0x285b71c1580>,
 'DatabaseB': <CybORG.Simulator.Host.Host at 0x285b71c1df0>,
 'DatabaseC': <CybORG.Simulator.Host.Host at 0x285b71c1eb0>,
 'Enterprise1_router': <CybORG.Simulator.Host.Host at 0x285b71e52e0>,
 'Enterprise2_router': <CybORG.Simulator.Host.Host at 0x285b71e5370>,
 'Enterprise3_router': <CybORG.Simulator.Host.Host at 0x285b71e5280>,
 'User_router': <CybORG.Simulator.Host.Host at 0x285b71e5730>,
 'Operation_router': <CybORG.Simulator.Host.Host at 0x2

In [5]:
from CybORG.Simulator.Actions import *

In [7]:
pprint(extension.host_absvul_map)
absvul1 = extension.host_absvul_map['API_Gateway']['SB1-CVE-2018-11776']
absvul2 = extension.host_absvul_map['ServiceB1']['SB2-CVE-2016-4461']
absvul3 = extension.host_absvul_map['ServiceB2']['SB1-CVE-2017-12611']
absvul4 = extension.host_absvul_map['ServiceB1']['DB-CVE-2017-12611']

{'API_Gateway': {'SA1-CVE-2018-11776': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002396D8F1580>,
                 'SB1-CVE-2018-11776': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002395D57F370>,
                 'SC1-CVE-2018-11776': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002395D57F6A0>,
                 'SC2-CVE-2018-11776': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002395D53CD60>},
 'ServiceA1': {'SA2-CVE-2016-4461': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002396D998F70>},
 'ServiceA2': {'DAC-CVE-2017-12611': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002396D998E20>},
 'ServiceB1': {'DB-CVE-2017-12611': <CybORG.Simulator.AbstractVulnerability.AbstractVulnerability object at 0x000002396D998EB0>,
               'SB2-CVE-2016-4461': <CybORG.Simulator.AbstractVulnerability.Abstr

In [8]:
results = env.reset(agent='Red')
# API_Gateway -> Discover ServiceB1
action1 = ExploitLocalVulnerability(session=0,agent='Red',target_host_id='API_Gateway',absvul=absvul1)
results = env.step(action=action1,agent='Red')
pprint(results.observation)

{'10.0.18.188': {'Interface': [{'IP Address': IPv4Address('10.0.18.188'),
                                'Subnet': IPv4Network('10.0.18.176/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


In [8]:
action7 = RollBackHost(session=0, agent='Blue', hostname='ServiceB1', host_absvul_map=extension.host_absvul_map)
results = env.step(action=action7,agent="Blue")
pprint(results.observation)

{'success': <TrinaryEnum.TRUE: 1>}


In [9]:
# ServiceB1 -> Discover ServiceB2
action2 = ExploitRemoteVulnerability(session=0,agent='Red',target_host_id='ServiceB1',absvul=absvul2)
results = env.step(action=action2,agent="Red")
pprint(results.observation)

{'10.0.18.182': {'Interface': [{'IP Address': IPv4Address('10.0.18.182'),
                                'Subnet': IPv4Network('10.0.18.176/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


In [11]:
action7 = RollBackHost(session=0, agent='Blue', hostname='ServiceB1')
results = env.step(action=action7,agent="Blue")
pprint(results.observation)

{'success': <TrinaryEnum.TRUE: 1>}


In [10]:
# ServiceB2 -> Exploit ServiceB1(Reverse Session)
action3 = ExploitRemoteVulnerability(session=0,agent='Red',target_host_id='ServiceB2',absvul=absvul3)
results = env.step(action=action3,agent="Red")
pprint(results.observation)

{'10.0.18.188': {'Interface': [{'IP Address': IPv4Address('10.0.18.188')}],
                 'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                 'local_address': IPv4Address('10.0.18.188'),
                                                 'local_port': 22}],
                                'Process Type': <ProcessType.SSH: 2>},
                               {'Connections': [{'local_address': IPv4Address('10.0.18.188'),
                                                 'local_port': 22,
                                                 'remote_address': IPv4Address('10.0.49.215'),
                                                 'remote_port': 49480}],
                                'Process Type': <ProcessType.SSH: 2>}],
                 'Sessions': [{'Agent': 'Red',
                               'ID': 1,
                               'Type': <SessionType.SSH: 2>,
                               'Username': 'vagrant'}],
  

In [9]:
# PrivilegeEscalate ServiceB1
action4 = PrivilegeEscalate(hostname='ServiceB1',session=0,agent='Red')
results = env.step(action=action4,agent="Red")
pprint(results.observation)

{'ServiceB1': {'Sessions': [{'Agent': 'Red',
                             'ID': 1,
                             'Type': <SessionType.SSH: 2>,
                             'Username': 'SYSTEM'}]},
 'success': <TrinaryEnum.TRUE: 1>}


In [10]:
# ServiceB1 -> Exploit DatabaseB(Reverse Session)
action5 = ExploitLocalVulnerability(session=0,agent='Red',target_host_id='ServiceB1',absvul=absvul4)
results = env.step(action=action5,agent="Red")
pprint(results.observation)

{'10.0.225.122': {'Interface': [{'IP Address': IPv4Address('10.0.225.122')}],
                  'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                  'local_address': IPv4Address('10.0.225.122'),
                                                  'local_port': 22}],
                                 'Process Type': <ProcessType.SSH: 2>},
                                {'Connections': [{'local_address': IPv4Address('10.0.225.122'),
                                                  'local_port': 22,
                                                  'remote_address': IPv4Address('10.0.39.82'),
                                                  'remote_port': 52764}],
                                 'Process Type': <ProcessType.SSH: 2>}],
                  'Sessions': [{'Agent': 'Red',
                                'ID': 2,
                                'Type': <SessionType.SSH: 2>,
                                'Username':

In [11]:
# PrivilegeEscalate DatabaseB
action6 = PrivilegeEscalate(hostname='DatabaseB',session=0,agent='Red')
results = env.step(action=action6,agent="Red")
pprint(results.observation)

{'DatabaseB': {'Sessions': [{'Agent': 'Red',
                             'ID': 2,
                             'Type': <SessionType.SSH: 2>,
                             'Username': 'SYSTEM'}]},
 'success': <TrinaryEnum.TRUE: 1>}
